<a href="https://colab.research.google.com/github/Sergey-Kiselev-dev/ML_sem_pub/blob/main/ML_01_sem_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class DummyClass:
    pass


dummy_object = DummyClass()

dummy_object


In [2]:
import math

vars(DummyClass)

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'DummyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'DummyClass' objects>,
              '__doc__': None})

In [6]:
Методы
Классы могут определять функции, которые можно будет вызывать от их объектов. Такие функции называются методами. В любом (почти) методе первым аргументом является self. В него будет передан сам объект, от которого вызывается метод.

Наш класс не особо полезен, так как ничего не делает. Давайте напишем класс с методом. Вызов методов происходит через оператор .

In [3]:
GOOSE = """
░░░░░▄▀▀▀▄░░░░░░░░
▄███▀░◐░░░▌░░░░░░░
░░░░▌░░░░░▐░░░░░░░
░░░░▐░░░░░▐░░░░░░░
░░░░▌░░░░░▐▄▄░░░░░
░░░░▌░░░░▄▀▒▒▀▀▀▀▄
░░░▐░░░░▐▒▒▒▒▒▒▒▒▀▀▄
░░░▐░░░░▐▄▒▒▒▒▒▒▒▒▒▒▀▄
░░░░▀▄░░░░▀▄▒▒▒▒▒▒▒▒▒▒▀▄
░░░░░░▀▄▄▄▄▄█▄▄▄▄▄▄▄▄▄▄▄▀▄
░░░░░░░░░░░▌▌▌▌░░░░░
░░░░░░░░░░░▌▌░▌▌░░░░░
░░░░░░░░░▄▄▌▌▄▌▌░░░░░"""


class GoosePrinter:
    GOOSE = ""

    def print_goose(self) -> None:
        print(self.GOOSE)


goose_printer = GoosePrinter()

goose_printer.print_goose()


In [4]:
vars(GoosePrinter)

mappingproxy({'__module__': '__main__',
              'GOOSE': '',
              'print_goose': <function __main__.GoosePrinter.print_goose(self) -> None>,
              '__dict__': <attribute '__dict__' of 'GoosePrinter' objects>,
              '__weakref__': <attribute '__weakref__' of 'GoosePrinter' objects>,
              '__doc__': None})

In [5]:
class Point2D:
    """
    A point in 2D space
    :param x: x coordinate
    :param y: y coordinate
    """

    def __init__(self, x: int, y: int) -> None:
        # Создаём атрибуты и присваиваем им переданные значения координат
        self.x = x
        self.y = y

    def print_coords(self) -> None:
        # Выводим координаты, обращаясь к ним через self с точкой
        print(f"({self.x}, {self.y})")

    def first_coord(self) -> int:
        return x


point = Point2D(3, 5)
point.print_coords()

vars(Point2D)

(3, 5)


mappingproxy({'__module__': '__main__',
              '__doc__': '\n    A point in 2D space\n    :param x: x coordinate\n    :param y: y coordinate\n    ',
              '__init__': <function __main__.Point2D.__init__(self, x: int, y: int) -> None>,
              'print_coords': <function __main__.Point2D.print_coords(self) -> None>,
              'first_coord': <function __main__.Point2D.first_coord(self) -> int>,
              '__dict__': <attribute '__dict__' of 'Point2D' objects>,
              '__weakref__': <attribute '__weakref__' of 'Point2D' objects>})

Magic методы
В питоне у классов могут быть так называемые магические методы. Это методы, которые может определить класс, которые позволяют ему действовать определённым образом. Обычно они не вызываются напрямую.

Все магические методы отличаются тем, что их названия начинаются и заканчиваются на двойные подчёркивания (__method__). С одним таким мы уже познакомились — это метод __init__, который вызывается при создании объекта класса. Давайте посмотрим, какие ещё бывают магические методы:

__str__ и __repr__ (Документация)
Методы __str__ и __repr__ позволяют добавить текстовое описание к объекту класса. Они возвращают строку, описывающую объект.

Метод __str__ должен возвращать строку, описывающую объект простым, понятным, читаемым образом, а метод __repr__ предназаначен больше для дебага и должен возвращать всю информацию об объекте, а его вывод должен в идеале быть исполняемым кодом, с помощью которого можно было бы создать такой же объект.

Метод __str__ вызывается, например, когда мы принтим объект или берём str от него (например, str(point)). Метод __repr__ вызывается, например, когда мы просто выводим объект в консоли или берём repr от него. Если __str__ не определён, по дефолту его роль будет играть __repr__.

Давайте избавимся от метода print_coords и добавим нашим точкам хорошее форматирование:

In [6]:
# Для начала убедимся, что по дефолту наши точки не умеют красиво печататься:
print(f"Некрасивая точка: {Point2D(5, 5)}")


class Point2D:
    """
    A point in 2D space
    :param x: x coordinate
    :param y: y coordinate
    """

    def __init__(self, x: int, y: int) -> None:
        # Создаём атрибуты и присваиваем им переданные значения координат
        self.x = x
        self.y = y

    def __repr__(self) -> str:
        return f"Point2D({self.x}, {self.y})"

    def __str__(self) -> str:
        return f"A 2D point with coordinates ({self.x}, {self.y})"

    def __deepcopy__(self):
        pass


# Посмотрим, что у нас есть теперь:
print(Point2D(5, 5))
Point2D(3, 4)

Некрасивая точка: <__main__.Point2D object at 0x78f6b5b2b8b0>
A 2D point with coordinates (5, 5)


Point2D(3, 4)

In [7]:
p = Point2D(10, 11)
p

Point2D(10, 11)

In [8]:
print(p)
p + Point2D(1, 1)

A 2D point with coordinates (10, 11)


TypeError: unsupported operand type(s) for +: 'Point2D' and 'Point2D'

Обратите внимание, что __str__ вызвался, когда мы запринтили объект, а __repr__, когда мы вывели его в конце ячейки.

__add__, __sub__, __mul__, __truediv__, etc. (Документация)
Методы __add__, __sub__, __mul__, __truediv__ позволяют добавить классу функционал сложения, вычитания, умножения, деления и так далее, (операторы +, -, *, /, etc.). Они вызываются от левого операнда и применяются к правому, возвращая результат. Давайте научим наши точки складываться и вычитаться:

In [9]:
class Point2D:
    """
    A point in 2D space
    :param x: x coordinate
    :param y: y coordinate
    """

    def __init__(self, x: int, y: int) -> None:
        # Создаём атрибуты и присваиваем им переданные значения координат
        self.x = x
        self.y = y

    def __repr__(self) -> str:
        return f"Point2D({self.x}, {self.y})"

    def __str__(self) -> str:
        return f"A 2D point with coordinates ({self.x}, {self.y})"

    def __add__(self, other: Point2D) -> Point2D:
        return Point2D(self.x + other.x, self.y + other.y)

    def __sub__(self, other: Point2D) -> Point2D:
        return Point2D(self.x - other.x, self.y - other.y)


print(Point2D(3, 5) + Point2D(4, 7))
print(Point2D(3, 5) - Point2D(4, 7))

A 2D point with coordinates (7, 12)
A 2D point with coordinates (-1, -2)


У каждого из вышеописанных методов есть так же версия "i" в начале, отвечающая за операцию с присвоением (__iadd__ — +=, __isub__ — -=, etc.)

В принципе, операции с присовением будут работать и так, питон сам выведет их из обычных операций. Но на самом деле они не будут модифицировать объект in-place, а будут возвращать новый объект и присваивать его переменной:

In [10]:
a = Point2D(3, 3)
print(id(a))

a += Point2D(1, 1)
print(id(a))
print(a)

133001005676048
133001005677536
A 2D point with coordinates (4, 4)


В данном случае это скорее правильное поведение, но давайте для практики добавим в наш класс эти методы:

In [12]:
class Point2D:
    """
    A point in 2D space
    :param x: x coordinate
    :param y: y coordinate
    """

    def __init__(self, x: int, y: int) -> None:
        # Создаём атрибуты и присваиваем им переданные значения координат
        self.x = x
        self.y = y

    def __repr__(self) -> str:
        return f"Point2D({self.x}, {self.y})"

    def __str__(self) -> str:
        return f"A 2D point with coordinates ({self.x}, {self.y})"

    def __add__(self, other: Point2D) -> Point2D:
        return Point2D(self.x + other.x, self.y + other.y)

    def __sub__(self, other: Point2D) -> Point2D:
        return Point2D(self.x - other.x, self.y - other.y)

    def __iadd__(self, other: Point2D) -> Point2D:
        self.x += other.x
        self.y += other.y
        return self

    def __isub__(self, other: Point2D) -> Point2D:
        self.x -= other.x
        self.y -= other.y
        return self

In [13]:
a = Point2D(3, 3)
print(id(a))

a += Point2D(2, 2)
print(id(a))
print(a)

a += Point2D(2, 2)
print(id(a))
print(a)

133001005677680
133001005677680
A 2D point with coordinates (5, 5)
133001005677680
A 2D point with coordinates (7, 7)


еперь при использовании += мы остаёмся с тем же объектом.

__eq__, __ne__, __lt__, __le__, __gt__, __ge__, (Документация)
По умолчанию объекты классов в питоне не имеют порядка и сравниваются на равенство при помощи is. Таким образом, две точки будут равны друг другу только тогда когда они представляют один и тот же объект в памяти:

In [14]:
a = Point2D(1, 1)
b = Point2D(1, 1)

a == b, a == a

(False, True)

Методы __eq__, __ne__, __lt__, __le__, __gt__, __ge__ позволяют задать правила сравнения для объектов класса.

Давайте научим наши точки сравниваться:

In [15]:
class Point2D:
    """
    A point in 2D space
    :param x: x coordinate
    :param y: y coordinate
    """

    def __init__(self, x: int, y: int) -> None:
        # Создаём атрибуты и присваиваем им переданные значения координат
        self.x = x
        self.y = y

    def __repr__(self) -> str:
        return f"Point2D({self.x}, {self.y})"

    def __str__(self) -> str:
        return f"A 2D point with coordinates ({self.x}, {self.y})"

    def __add__(self, other: Point2D) -> Point2D:
        return Point2D(self.x + other.x, self.y + other.y)

    def __sub__(self, other: Point2D) -> Point2D:
        return Point2D(self.x - other.x, self.y - other.y)

    def __iadd__(self, other: Point2D) -> Point2D:
        self.x += other.x
        self.y += other.y
        return self

    def __isub__(self, other: Point2D) -> Point2D:
        self.x -= other.x
        self.y -= other.y
        return self

    def __eq__(self, other: Point2D) -> Point2D:
        return self.x == other.x and self.y == other.y

    def __del__(self):
        print("delete", end=" ")
        print(self)


# print(Point2D(1, 1) == Point2D(1, 1))
# print(Point2D(1, 1) != Point2D(1, 1))
# print(Point2D(1, 1) == Point2D(1, 2))
# print(Point2D(1, 1) != Point2D(1, 2))

p = Point2D(1, 1)
del p

delete A 2D point with coordinates (1, 1)


delete A 2D point with coordinates (1, 1)
Обратите внимание, что когда мы определили __eq__ (==) для сравнения, __ne__ (!=) вывелся автоматически. Однако, это исключение. Если мы, например, определим ещё и __gt__ (>), операция __le__ (<=) сама не выведется.

NB: Если вам хочется написать одну операцию, чтобы остальные вывелись автоматически, используйте декоратор functools.total_ordering

Кроме описанных в питоне есть ещё великое множество магических методов, которые способны осуществить примерно любое поведение, которое вы видели в питоне. Все их можно посмотреть в документации или нагуглить.

Из полезных можно отметить:

__new__ и __del__ — создание и удаление объекта.
__len__ — возвращение длины объекта (например, контейнера). Используется функцией len.
__getitem__, __setitem__ — индексация квадратными скобками.
__getattr__, __setattr__ — обращение к атрибутам по точке.
__iter__ — возвращает итератор, проходящий по объекту. Используется, например, for.
__next__ — возвращает следующее состояние итератора
__nonzero__ — определяет поведение функции bool на объекте.
__contains__ — определяет поведение оператора in (полезно для контейнеров)
__call__ — вызывается, когда объект класса вызывается со скобками (как функция). Позволяет сделать объекты класса callable.
__copy__, __deepcopy__ — определяют, как объект класса копируется.
Копирование
Переменные в питоне всегда хранят ссылку на объект. Если мы присвоим объект нашего класса двум разным переменным, они обе будут хранить один и тот же объект и "меняться" вместе:

In [16]:
a = Point2D(3, 4)
print(a)

b = a
b.x = -1

print(a)

print(id(a), id(b))

A 2D point with coordinates (3, 4)
A 2D point with coordinates (-1, 4)
133000979304880 133000979304880


Мы поменяли значение в переменной b, но значение в a тоже поменялось, потому что они ссылаются на один и тот же объект.

Чтобы избежать этого недоразумения в питоне существует модуль copy и соответствующая функция:

In [17]:
import copy

a = Point2D(3, 4)
print(a)

b = copy.copy(a)
b.x = -1

print(a)
print(b)

A 2D point with coordinates (3, 4)
A 2D point with coordinates (3, 4)
A 2D point with coordinates (-1, 4)


Однако и это не спасёт нас, если в объекте нашего класса снова хранятся изменяемые переменные (например, списки). В таком случае нам понадобится функция deepcopy, которая полностью скопирует как сам объект класса, так и всё внутри него, по сути действуя рекурсивно.

Продемонстрируем это на новом классе Student:

In [18]:
import copy
from typing import Iterable


class Student:
    """
    A class representing student along with his name and classes he/she takes
    :param name: name of the student
    :param classes: iterable of strings with names of classes
    """

    def __init__(self, name: str, classes: Iterable[str]) -> None:
        self.name = name
        self.classes = classes

    def __repr__(self) -> str:
        return f"Student({repr(self.name)}, {repr(self.classes)})"


student = Student("Ваня", ["Линал", "Алгосы", "Машинное обучение"])
student_deepcopy = copy.deepcopy(student)
student_copy = copy.copy(student)
student_naive = student

# Распечатаем студента
print("До изменений")
print(student)

student_deepcopy.name = "Катя"
student_deepcopy.classes[0] = "Матан"
print("\nИзменения в deepcopy")
print(f"{student_deepcopy}")
print(f"{student}")

student_copy.name = "Лиза"
student_copy.classes[0] = "Экономика"
print("\nИзменения в copy")
print(f"{student_copy}")
print(f"{student}")

student_naive.name = "Лёша"
student_naive.classes[0] = "Алгебра"
print("\nИзменения в простом присваивании")
print(f"{student_naive}")
print(f"{student}")

До изменений
Student('Ваня', ['Линал', 'Алгосы', 'Машинное обучение'])

Изменения в deepcopy
Student('Катя', ['Матан', 'Алгосы', 'Машинное обучение'])
Student('Ваня', ['Линал', 'Алгосы', 'Машинное обучение'])

Изменения в copy
Student('Лиза', ['Экономика', 'Алгосы', 'Машинное обучение'])
Student('Ваня', ['Экономика', 'Алгосы', 'Машинное обучение'])

Изменения в простом присваивании
Student('Лёша', ['Алгебра', 'Алгосы', 'Машинное обучение'])
Student('Лёша', ['Алгебра', 'Алгосы', 'Машинное обучение'])
